<a href="https://colab.research.google.com/github/dhingratul/generative-ai-with-llm/blob/main/vector-db/pinecone_fast_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Requirements
!pip install pinecone-client~=2.2.2
!pip install python-dotenv~=1.0.0
!pip install fastapi~=0.97.0
!pip install uvicorn~=0.22.0
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19866 sha256=6da874321f1c8a25972759673e8aeb631a4f4fcae22930d048c8c348fd6eb825
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [3]:
import pinecone
import os
import json
from dotenv import load_dotenv, find_dotenv

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:

class PineconeOperations:

    def __init__(self):
        _ = load_dotenv(find_dotenv())  # read local .env file
        api_key = os.getenv('b')
        api_env = os.getenv('PINECONE_ENVIRONMENT')

        pinecone.init(
            api_key=api_key,
            environment=api_env
        )
        self.index = None

    def create_index(self, index_name='default') -> list:
        # fetch the list of indexes
        indexes = pinecone.list_indexes()

        # create index if there are no indexes found
        if len(indexes) == 0:
            pinecone.create_index(index_name, dimension=8, metric="euclidean")

        return indexes

    def connect_index(self):
        indexes = self.create_index()
        # connect to a specific index
        self.index = pinecone.Index(indexes[0])

    def upsert(self, data):
        # sample data of the format
        # [
        #     ("A", [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
        #     ("B", [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]),
        #     ("C", [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]),
        #     ("D", [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4]),
        #     ("E", [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
        # ]
        # Upsert sample data (5 8-dimensional vectors)
        return json.loads(str(self.index.upsert(vectors=data, namespace="quickstart")).replace("'", '"'))

    def fetch_stats(self):
        # fetches stats about the index
        stats = self.index.describe_index_stats()
        return str(stats)

    def query(self, query_vector):
        # query from the index, eg: [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
        response = self.index.query(
            vector=query_vector,
            top_k=2,
            include_values=True,
            namespace="quickstart"
        )
        return json.loads(str(response).replace("'", '"'))

In [8]:
from fastapi import FastAPI
from typing import List, Tuple, Any
from pydantic.main import BaseModel

app = FastAPI()
pineconeOps = PineconeOperations()


class Data(BaseModel):
    payload: List[Tuple[Any, Any]]


@app.get("/api/v1/health")
async def root():
    return {"message": "OK"}


@app.post("/api/v1/index")
async def create_index(name: str):
    return pineconeOps.create_index(index_name=name)


@app.get("/api/v1/index/stats")
async def stats():
    return pineconeOps.fetch_stats()


@app.get("/api/v1/connect")
async def create_index():
    return pineconeOps.connect_index()


@app.post("/api/v1/vectors")
async def create_index(data: Data):
    return pineconeOps.upsert(data=data.payload)


@app.post("/api/v1/search-vector")
async def create_index(payload: List[Any]):
    return pineconeOps.query(query_vector=payload)


In [15]:
from uvicorn import Config, Server

config = Config(app)
server = Server(config=config)
await server.serve()

INFO:     Started server process [284]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [284]
